### Setup

#### Households
Utility $u(c)$ is CRRA <br>
Stochastic process for (log) income is an AR(1)<br>
Key equations:<br>


$\begin{align} 
% V_t(a,y) = \max_{c, a'} &\left\{\frac{c^{1-\sigma}}{1-\sigma} + \beta \sum_{y'}V_{t+1}(a', y') \Gamma(y,y') \right\} \\
\tag{Euler} u_c(c) &\geq \beta (1+r) \mathbb{E}[u_c(c')]  \\
\tag{Budget} c + a' &= (1 + r)a + y \\
\tag{Borrowing limit} a' &\geq 0
\end{align}$

#### Firms
$\begin{align} \notag
r + \delta = \alpha K^{\alpha-1} L^{1-\alpha} \Rightarrow K &= \left( \frac{\alpha}{r+\delta} \right)^{\frac{1}{1-\alpha}} L  \\ \notag
w &= (1-\alpha) K^{\alpha} L^{-\alpha}
\end{align}$

### Start

In [1]:
using Parameters
using Interpolations
using Plots
using Setfield

In [2]:

@with_kw struct ModelParameters{T} # Economic Variables
    α :: T = 1.0/3.0
    β :: T = 0.93
    δ :: T = 0.1
    γ :: T = 2.0
end 

@with_kw struct EqParameters{T} # Equilibrium Variables
    r :: T = 0.03   # interest rate
    w :: T = 1.0    # wage
    K :: T = ((r + m_par.δ)/(m_par.α))^(1/(m_par.α-1)) # capital
end 

@with_kw struct NumericalParameters # Numerical Variables
    na   :: Int = 200 # asset grid
    nadist   :: Int = na*3 # asset grid
    amin :: Float64 = 0.0 
    amax :: Float64 = 50.0 
    ny   :: Int = 5 # income grid
    tol_pol :: Float64 = 1e-6
    tol_dist :: Float64 = 1e-10
end

# Load it already here, as it is needed below
n_par= NumericalParameters()

@with_kw struct ContainerHA{T} # T= Array{Float64,1}
    A  :: T  = zeros(n_par.na, n_par.ny) # policy function assets
    C  :: T  = zeros(n_par.na, n_par.ny) # policy function consumption
end

@with_kw struct Grids
    gridA :: Array{Float64,1} = zeros(n_par.na)
    Π :: Array{Float64,2} = zeros(n_par.ny, n_par.ny)       # transition matrix income
    gridY :: Array{Float64,1} = zeros(n_par.ny)   # income grid
end


Grids

#### HA Functions

In [56]:
function egm(r,w, m_par::ModelParameters, n_par::NumericalParameters, grids::Grids, gV_conHA::ContainerHA)

    # Unpack
    @unpack_ModelParameters m_par
    @unpack_NumericalParameters n_par
    @unpack_Grids grids
    @unpack_ContainerHA gV_conHA

    Ai= copy(A)
    difference= 1000.0
    it=0
    Uc= zeros(na,ny)

    while difference > tol_pol && it < 1000

        for y in 1:ny
            Uc[:,y]= ( (1+r).*gridA .+ w*gridY[y] .- A[:,y] ).^(-γ)
        end
        EUc= Uc * Π' # expectation (* is matrix multiplication)
        RHS= β * (1+r) .* EUc # RHS of Euler
        for y in 1:ny
            Cendog= RHS[:,y].^(-1/γ)
            Aendog= (gridA .+ Cendog .- w*gridY[y]) ./ (1 + r)
            itp= LinearInterpolation(Aendog, gridA, extrapolation_bc=Line())
            Ai[:,y]= itp.(gridA)
        end        
        Ai[Ai.<=amin] .= amin # set borrowing constraint
        difference= maximum(abs.(A .- Ai))
        A= copy(Ai)

        it= it+1

    end 

        # Back out consumption policy
        for y in 1:ny
            C[:,y]= (1+r).*gridA .+ w*gridY[y] .- A[:,y]
        end

    gV_conHA= ContainerHA(A=A, C=C)

    # println("Finished EGM in iteration = ", it, ", and diff = ", difference)

    return gV_conHA

end

function weights_and_indices(A,gridA,na,ny)

    A_indices= zeros(Int, na, ny) 
    wei= zeros(Float64,na,ny) 

    for y in 1:ny
        # Indices
        A_indices[:,y] = searchsortedlast.(Ref(gridA), A[:,y])  # gives left bracket; 0 if below grid, na if above grid
    end
    A_indices[A_indices .== 0] .= 1
    A_indices[A_indices .== na] .= na-1
    for y in 1:ny
        wei[:,y]= (A[:,y] .- [gridA[i] for i in A_indices[:,y]]) ./ ([gridA[i+1] for i in A_indices[:,y]] .- [gridA[i] for i in A_indices[:,y]])
    end
    # should never be binding, but just to be save
    wei[wei.>1] .=1
    wei[wei.<0] .=0

    return wei, A_indices

end


function loop!(A_disti::Array{Float64,2},A_indices::Array{Int,2},wei::Array{Float64,2},Π::Array{Float64,2},A_dist::Array{Float64,2},na::Int,ny::Int)

    @inbounds for i in 1:na
        for s in 1:ny
            index= A_indices[i,s]
            lottery1= (1-wei[i,s]) * A_dist[i,s]
            lottery2= wei[i,s]* A_dist[i,s]
            for si in 1:ny
                A_disti[index,si] += lottery1 * Π[s,si]
                A_disti[index + 1,si] += lottery2 * Π[s,si] 
            end
        end
    end

end

function density_discretization(gV_conHA::ContainerHA, grids::Grids, n_par::NumericalParameters,A_dist)

    # Unpack
    @unpack_ContainerHA gV_conHA
    @unpack_NumericalParameters n_par
    @unpack_Grids grids
    
    # retrieve indices and weights for lottery
    wei, A_indices= weights_and_indices(A, gridA, na, ny)

    difference= 100.0
    it=1
    # iterate over discretized density to get fixed point
    while difference > 1e-10 && it < 10000
    
        # always pre-allocate to zero
        A_disti= zeros(Float64,na,ny) 

        loop!(A_disti,A_indices,wei,Π,A_dist,na,ny)

        difference= maximum(abs.(A_disti .- A_dist))
        A_dist= A_disti ./ sum(A_disti)
        it= it+1

    end

    return A_dist

end

density_discretization (generic function with 1 method)

To save on code, I use a grid and transition matrix from Hintermaier \& Koeniger (2010)

In [57]:
incomeGrid= [0.09, 0.39, 0.74, 1.22, 2.57];
transitionMatrix= [0.9854 0.0146 0 0 0; 
                    0.0045 0.8451 0.1491 0.0013 0; 
                    0 0.1359 0.6787 0.1843 0.0011; 
                    0 0.0029 0.2208 0.6963 0.0800; 
                    0 0 0.0006 0.1455 0.8539];

function get_Invariant_Distribution(Π::Array{Float64,2}, n::Int; tol= 1e-12)

    difference= 100.0

    distr= ones(n)/n # start from uniform

    while difference > tol
        distri= Π' * distr
        difference= maximum(abs.(distr .- distri))
        distr= copy(distri)
    end

    return distr

end

stationaryDistribution= get_Invariant_Distribution(transitionMatrix, 5);

#### Set up containers

In [58]:
m_par= ModelParameters{Float64}()
gV_conHA= ContainerHA{Array{Float64,2}}();
eq_par= EqParameters();

gridA= n_par.amin .+ (collect(0:n_par.na-1)./n_par.na).^2 .* (n_par.amax - n_par.amin);
grids= Grids(gridA=gridA, Π=transitionMatrix, gridY=incomeGrid);


Note that $L^{supply}$ is not equal to 1. 

In [59]:
println("Labor supply equals = ", Lsupply)


Labor supply equals = 0.9520370924645079


### Aiyagari loop

In [60]:
# Guess
Cguess= ones(n_par.na, n_par.ny) .* eq_par.r .+ gridA;
gV_conHA= ContainerHA(C=Cguess);

# Start from uniform distribution
mA_dist= ones(n_par.na, n_par.ny) ./ (n_par.na*n_par.ny); # start from uniform

# Initialize
res= 10.0
iteration= 0

while res > 1e-6

### FIRM 
# Update capital
K = ((m_par.α)/(eq_par.r + m_par.δ))^(1/(1-m_par.α)) * Lsupply
# Update wage
w = (1 - m_par.α)*K^m_par.α * Lsupply^(-m_par.α)


### HOUSEHOLDS
# Get policy functions
gV_conHA= egm(eq_par.r, w, m_par, n_par, grids, gV_conHA);
# Get invariant distribution
mA_dist= density_discretization(gV_conHA, grids, n_par, mA_dist);

### Market clearing
# Capital supply / Asset demand
aggA = sum(gV_conHA.A .* mA_dist);

# Check market clearing
res= abs(aggA - K);

if iteration % 10 == 0
    println("Residual on market clearing: ", res)
end

# Update interest rate
rnew = eq_par.r - 0.005*(aggA - K)
eq_par= @set! eq_par.r = rnew
iteration=iteration + 1

end # end while

# Update final 
eq_par = @set! eq_par.K = ((m_par.α)/(eq_par.r + m_par.δ))^(1/(1-m_par.α)) * Lsupply;
eq_par = @set! eq_par.w = (1 - m_par.α)*eq_par.K^m_par.α * Lsupply^(-m_par.α);


Residual on market clearing: 0.7004912149863891
Residual on market clearing: 4.343394388151012e-6
Residual on market clearing: 4.157175865771023e-6


In [63]:
println("Equilibrium = ", eq_par)

K= eq_par.K
Y= K^m_par.α * Lsupply^(1-m_par.α)
C= sum(gV_conHA.C .* mA_dist)
I= m_par.δ * K
A= sum(gV_conHA.A .* mA_dist)

println("Goods market clearing: ", Y - C - I)
println("Asset market clearing: ", A - K)


Equilibrium = EqParameters{Float64}
  r: Float64 0.025276119913335955
  w: Float64 1.0874616911442323
  K: Float64 4.132087852496143

Goods market clearing: 1.1369152619344902e-8
Asset market clearing: -3.987531833615776e-7


In [64]:
# HH BC
for y in 1:n_par.ny
    println(minimum(abs.(gV_conHA.C[:,y] .+ gV_conHA.A[:,y] .- eq_par.w * grids.gridY[y] + (1+eq_par.r)*grids.gridA )))
end

1.034798829957495e-9
4.48412823539357e-9
8.508345805680051e-9
1.4027272898786691e-8
2.9549255486926995e-8
